In [44]:
import json 
import copy

In [6]:
!ls

CLEVR_test_questions.json  CLEVR_val_questions.json
CLEVR_train_questions.json


In [64]:
 %cd questions 
!ls

/Users/lisazhao/Desktop/CLEVR_v1.0
/Users/lisazhao/Desktop/CLEVR_v1.0/questions
CLEVR_test_questions.json  CLEVR_val_questions.json
CLEVR_train_questions.json


In [65]:
with open('CLEVR_train_questions.json') as f:
    q_dict = json.load(f)
    
print('done q_dict')

done q_dict


In [9]:
%cd .. 
%cd scenes
!ls

/Users/lisazhao/Desktop/CLEVR_v1.0
/Users/lisazhao/Desktop/CLEVR_v1.0/scenes
CLEVR_train_scenes.json CLEVR_val_scenes.json


In [10]:
with open('CLEVR_train_scenes.json') as f:
    scene_dict = json.load(f)
    
print('done scene_dict')

done scene_dict


In [11]:
print(len(q_dict['questions']))
print(len(scene_dict['scenes']))

699989
70000


In [26]:
q_dict['questions'][6]

{'image_index': 0,
 'program': [{'inputs': [], 'function': 'scene', 'value_inputs': []},
  {'inputs': [0], 'function': 'filter_color', 'value_inputs': ['brown']},
  {'inputs': [1], 'function': 'filter_shape', 'value_inputs': ['sphere']},
  {'inputs': [2], 'function': 'unique', 'value_inputs': []},
  {'inputs': [3], 'function': 'relate', 'value_inputs': ['left']},
  {'inputs': [4], 'function': 'filter_size', 'value_inputs': ['large']},
  {'inputs': [5], 'function': 'filter_color', 'value_inputs': ['brown']},
  {'inputs': [6], 'function': 'filter_material', 'value_inputs': ['metal']},
  {'inputs': [7], 'function': 'unique', 'value_inputs': []},
  {'inputs': [8], 'function': 'relate', 'value_inputs': ['right']},
  {'inputs': [9], 'function': 'filter_color', 'value_inputs': ['brown']},
  {'inputs': [10], 'function': 'unique', 'value_inputs': []},
  {'inputs': [11], 'function': 'query_shape', 'value_inputs': []}],
 'question_index': 6,
 'image_filename': 'CLEVR_train_000000.png',
 'question

In [60]:
scene_dict['scenes'][0]

{'image_index': 0,
 'objects': [{'color': 'blue',
   'size': 'large',
   'rotation': 269.8517172617167,
   'shape': 'cube',
   '3d_coords': [-1.3705521821975708, 2.0794010162353516, 0.699999988079071],
   'material': 'rubber',
   'pixel_coords': [269, 88, 12.661545753479004]},
  {'color': 'green',
   'size': 'large',
   'rotation': 292.2219458666971,
   'shape': 'cylinder',
   '3d_coords': [-2.9289753437042236, -1.7488206624984741, 0.699999988079071],
   'material': 'metal',
   'pixel_coords': [93, 108, 11.522202491760254]},
  {'color': 'cyan',
   'size': 'small',
   'rotation': 25.545135239473026,
   'shape': 'cube',
   '3d_coords': [1.5515961647033691, 0.6776641607284546, 0.3499999940395355],
   'material': 'rubber',
   'pixel_coords': [319, 162, 10.045343399047852]},
  {'color': 'brown',
   'size': 'large',
   'rotation': 327.3489188814305,
   'shape': 'cylinder',
   '3d_coords': [-0.25301405787467957, -2.3089325428009033, 0.699999988079071],
   'material': 'metal',
   'pixel_coords

In [56]:
print(q_dict['questions'][0].keys())
print(scene_dict['scenes'][0].keys())

dict_keys(['image_index', 'program', 'question_index', 'image_filename', 'question_family_index', 'split', 'answer', 'question'])
dict_keys(['image_index', 'objects', 'relationships', 'image_filename', 'split', 'directions'])


In [39]:
function_holder = set() 

for idx, q in enumerate(q_dict['questions']): 
    if idx % 100000 == 0: 
        print(idx)
    for i in q['program']: 
        function_holder.add(i['function'])
        
print(len(function_holder))
function_holder

0
100000
200000
300000
400000
500000
600000
26


{'count',
 'equal_color',
 'equal_integer',
 'equal_material',
 'equal_shape',
 'equal_size',
 'exist',
 'filter_color',
 'filter_material',
 'filter_shape',
 'filter_size',
 'greater_than',
 'intersect',
 'less_than',
 'query_color',
 'query_material',
 'query_shape',
 'query_size',
 'relate',
 'same_color',
 'same_material',
 'same_shape',
 'same_size',
 'scene',
 'union',
 'unique'}

In [139]:
# input: 
# qu_dict with keys : dict_keys(['image_index', 'program', 'question_index', 
# 'image_filename', 'question_family_index', 'split', 'answer', 'question'])
# s_dict with keys: dict_keys(['image_index', 'objects', 'relationships', 
# 'image_filename', 'split', 'directions'])
# 
# output: 
# list of idx of relevant objects. 
# Idx agrees with the dict in scene.dict['scene'] that refers to this image 

def query_relevant_obj(qu_dict, s_dict):
    functions = [q['function'] for q in qu_dict['program']]
    
    # two special cases: union, intersection -- there must be two scenes before 
    if 'union' not in functions and 'intersection' not in functions:  
        scene_locs = getIndexPositions(functions, 'scene') + [len(functions)]
        for idx in range(len(scene_locs)-1):
            start_idx = scene_locs[idx]+1; end_idx = scene_locs[idx + 1]  
            scene_qu_dict = qu_dict['program'][start_idx: end_idx]
        
            # for this scene 
            objs = per_scene_get_obj(scene_qu_dict, s_dict)
                

                
    
def per_scene_get_obj(scene_qu_dict, s_dict): 
    functions = [q['function'] for q in scene_qu_dict]
    value_inputs = [q['value_inputs'] for q in scene_qu_dict]
    # initialize with all objs in image 
    rel_objects = list(range(len(s_dict['objects'])))
    
    for f, val in zip(functions, value_inputs): 
        if 'filter_' in f:
            keyword = f.partition("filter_")[2]
            rel_objects = filter_by_property(s_dict['objects'], keyword, val[0]) 
            
        if f == 'unique': 
            assert len(rel_objects) == 1 
            
        if 'same_' in f: 
            keyword = f.partition("same_")[2]
            rel_objects = filter_by_same(s_dict['objects'], keyword, rel_objects) 
            
        if f == 'relate':
            filter_by_relate(s_dict, val[0], rel_objects)
            
        if f == 'count':
            
    
    
    
def filter_by_property(s_dict_objs, keyword, target): 
    out = [] 
    for idx in range(len(s_dict_objs)):
        if s_dict_objs[idx][keyword] == target: 
            out.append(idx) 
    return out 
    
    
    
def filter_by_same(s_dict_objs, keyword, rel_objects): 
    # there must be only 1 obj in rel_objects since unique is always used before same_?
    match_idx = rel_objects[0]  
    target_feature = s_dict_objs[match_idx][keyword]
    
    out = [] 
    for idx in range(len(s_dict_objs)):
        # not including original object with this feature 
        if s_dict_objs[idx][keyword] == target_feature and idx != match_idx: 
            out.append(idx) 
    
    return out 
    
    
    
def filter_by_relate(s_dict, direction, rel_objects):
    # there must be only 1 obj in rel_objects since unique is always used before relate 
    match_idx = rel_objects[0]  
    return s_dict['relationships'][direction][match_idx]
    
    
def getIndexPositions(listOfElements, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    indexPosList = []
    indexPos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            indexPos = listOfElements.index(element, indexPos)
            # Add the index position in list
            indexPosList.append(indexPos)
            indexPos += 1
        except ValueError as e:
            break
    return indexPosList

In [136]:
q_dict['questions'][:40]

[{'image_index': 0,
  'program': [{'inputs': [], 'function': 'scene', 'value_inputs': []},
   {'inputs': [0], 'function': 'filter_size', 'value_inputs': ['large']},
   {'inputs': [1], 'function': 'filter_color', 'value_inputs': ['green']},
   {'inputs': [2], 'function': 'count', 'value_inputs': []},
   {'inputs': [], 'function': 'scene', 'value_inputs': []},
   {'inputs': [4], 'function': 'filter_size', 'value_inputs': ['large']},
   {'inputs': [5], 'function': 'filter_color', 'value_inputs': ['purple']},
   {'inputs': [6], 'function': 'filter_material', 'value_inputs': ['metal']},
   {'inputs': [7], 'function': 'filter_shape', 'value_inputs': ['cube']},
   {'inputs': [8], 'function': 'count', 'value_inputs': []},
   {'inputs': [3, 9], 'function': 'greater_than', 'value_inputs': []}],
  'question_index': 0,
  'image_filename': 'CLEVR_train_000000.png',
  'question_family_index': 2,
  'split': 'train',
  'answer': 'yes',
  'question': 'Are there more big green things than large purple s

In [14]:
# holder = [] 
# h = 0 

# for i in q_dict['questions']:
#     if h >= 100: 
#         print('hi')
#         break 
#     this_index = i['question_family_index']
#     if this_index in holder: 
#         h += 1 
#         continue
#     else: 
#         h = 0 
#         holder.append(this_index)

# sorted(holder)

In [57]:
# index = 1 
# print('')
# print('question_family_index =', index)
# print('')
# counter = 0 

# for i in q_dict['questions']: 
#     if i['question_family_index'] == index: 
#         print(i['question'])
#         counter += 1 
#         if counter == 5: 
#             index += 1 
#             counter = 0 
#             print('')
#             print('question_family_index = ', index)
#             print('')
            


In [58]:
# for i in q_dict['questions']: 
#     if i['question_family_index'] == 13: 
#         print(i['question'])